In [4]:
# CNN모델 만드는 방법

import torch
import torch.nn as nn

# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)
print(inputs.shape)

# 합성곱층과 풀링 선언하기
conv1 = nn.Conv2d(1, 32, 3, padding=1) # 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1
print(conv1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)
pool = nn.MaxPool2d(2)  # 정수하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정
print(pool)

# 구현체를 연결하여 모델 만들기
out = conv1(inputs)
print(out.shape)  # 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존됨
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)

# .view()를 사용해 텐서 펼치는 작업 
out = out.view(out.size(0), -1)  # 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
print(out.shape)

# 10개 차원의 텐서로 변환
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 1, 28, 28])
Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
torch.Size([1, 10])


In [10]:
# CNN으로 MNIST 분류하기

'''
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
'''

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

# DataLoader로 배치 크기 지정
data_loader = torch.utils.data.DataLoader(dataset= mnist_train, 
                         batch_size=batch_size, 
                         shuffle=True,
                        drop_last=True)

# CNN 모델 설계하기
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        # 모든 weights를 0으로 초기화하면 같은 출력값, 같은 gradient를 얻어 weight를 똑같이 업데이트함
        # Conv2d는 따로 initialization 방법을 정해주지 않을 경우 Xavier initialization 방법을 사용
        # 각 layer의 종류마다 다른 dafault initialization 방법을 선택함
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # FC layer를 위한 flatten
        out = self.fc(out)
        return out
    
# CNN 모델 정의
model = CNN().to(device)

# cost, optimizer 정의
cost = torch.nn.CrossEntropyLoss().to(device)  # 소프트맥스 함수 포함되어져 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 총 배치의 수 출력해보기
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
# 총 배치수 (600) * 배치 크기 (100) = 60000개 훈련데이터

# 모델 훈련시키기
for epoch in range(training_epochs):
    avg_cost = 0
    
    # 미니 배치 단위로 꺼내오기
    for data, targets in data_loader:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        y_pred = model(data)
        loss = cost(y_pred, targets)
        
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    

# 테스트
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

총 배치의 수 : 600
[Epoch:    1] cost = 0.223857522
[Epoch:    2] cost = 0.0621518306
[Epoch:    3] cost = 0.0448401198
[Epoch:    4] cost = 0.0354862362
[Epoch:    5] cost = 0.0291029662
[Epoch:    6] cost = 0.0247983951
[Epoch:    7] cost = 0.0208388809
[Epoch:    8] cost = 0.0181976575
[Epoch:    9] cost = 0.0149142751
[Epoch:   10] cost = 0.0127911828
[Epoch:   11] cost = 0.0101653524
[Epoch:   12] cost = 0.0100674033
[Epoch:   13] cost = 0.00852919463
[Epoch:   14] cost = 0.00702021411
[Epoch:   15] cost = 0.00650832709


C:\Users\Networking_Lab_009\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Networking_Lab_009\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9869999885559082


In [ ]:
# 깊은 CNN으로 MNIST 분류하기

'''
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 64, out_channel = 128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2, padding=1))

# 4번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 4 × 4 × 128 → batch_size × 2048
전결합층(뉴런 625개) + 활성화 함수 ReLU

# 5번 레이어 : 전결합층(Fully-Connected layer)
전결합층(뉴런 10개) + 활성화 함수 Softmax
'''

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

# DataLoader로 배치 크기 지정
data_loader = torch.utils.data.DataLoader(dataset= mnist_train, 
                         batch_size=batch_size, 
                         shuffle=True,
                        drop_last=True)

# CNN 모델 설계하기
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
    
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
            
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p= 1-self.keep_prob))  # dropout 적용
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # FC layer를 위한 flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out
    
# CNN 모델 정의
model = CNN().to(device)

# cost, optimizer 정의
cost = torch.nn.CrossEntropyLoss().to(device)  # 소프트맥스 함수 포함되어져 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # adam 사용

# 총 배치의 수 출력해보기
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
# 총 배치수 (600) * 배치 크기 (100) = 60000개 훈련데이터

# 모델 훈련시키기
for epoch in range(training_epochs):
    avg_cost = 0
    
    # 미니 배치 단위로 꺼내오기
    for data, targets in data_loader:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        y_pred = model(data)
        loss = cost(y_pred, targets)
        
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    

# 테스트
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())